In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [26]:
df_eval = pd.read_csv('../data/Evaluation_Set/RFMiD_Validation_Labels.csv')

In [27]:
df_train = pd.read_csv('../data/Training_Set/RFMiD_Training_Labels.csv')

In [28]:
df_train.head()

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Creating a different df to store disease risk column for the data

In [29]:
df_disease_risk_train = df_train['Disease_Risk']
df_disease_risk_eval = df_eval['Disease_Risk']

In [30]:
df_train.drop(['Disease_Risk'], axis=1, inplace=True)
df_eval.drop(['Disease_Risk'], axis=1, inplace=True)

In [31]:
df_train.head()

,ID,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,LS,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df_eval.head()

,ID,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,LS,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
0,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
## making ID column as index
df_train.set_index('ID', inplace=True)
df_eval.set_index('ID', inplace=True)

In [34]:
df_train = df_train[['DR', 'MH', 'TSLN', 'ODC']]
df_eval = df_eval[['DR', 'MH', 'TSLN', 'ODC']]

print("shape of train data: ", df_train.shape)
print("shape of eval data: ", df_eval.shape)

shape of train data:  (1920, 4)
shape of eval data:  (640, 4)


In [37]:
## removing the rows which have no diseases
df_train = df_train[df_train.sum(axis=1) != 0]
df_eval = df_eval[df_eval.sum(axis=1) != 0]

In [38]:
print("shape of train data: ", df_train.shape)
print("shape of eval data: ", df_eval.shape)

shape of train data:  (949, 4)
shape of eval data:  (295, 4)


In [39]:
df_train.head()

,DR,MH,TSLN,ODC
ID,,,,
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,0,1,0,1
5,1,0,0,0


In [40]:
df_eval.head()

,DR,MH,TSLN,ODC
ID,,,,
1,1,0,1,0
11,1,0,0,0
12,1,0,0,0
20,1,0,0,0
23,0,0,1,0


In [41]:
## adding the disease risk column to the train and eval data with ID as index
df_train['Disease_Risk'] = df_disease_risk_train
df_eval['Disease_Risk'] = df_disease_risk_eval

In [51]:
import cv2
import os
import numpy as np
import pandas as pd
## running a loop on the file to read the images and store them in a list
folder_path = '../data/Training_Set/Training'
images = []
for filename in os.listdir(folder_path):
    img = cv2.imread(os.path.join(folder_path, filename))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, heirarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(largest_contour)
    eyeball_roi = img[y:y+h, x:x+w]
    eyeball_roi = cv2.resize(eyeball_roi, (224, 224))
    cv2.imwrite(os.path.join('../data/Training_set_processed', filename), eyeball_roi)

In [1]:
import cv2

# Load the image
img = cv2.imread('../data/Training_set_processed/1.png')

# Set the center and radius of the circle around the eyeball
center = (x, y)  # x and y are obtained from cv2.boundingRect(largest_contour)
radius = r  # r is the radius of the largest contour

# Create a black mask with the same size as the image
mask = np.zeros_like(img)

# Draw a white circle on the mask
cv2.circle(mask, center, radius, (255, 255, 255), -1)

# Apply the mask to the image
masked_img = cv2.bitwise_and(img, mask)

# Find the bounding box of the circle
nonzero_pixels = cv2.findNonZero(mask)
x, y, w, h = cv2.boundingRect(nonzero_pixels)

# Crop the image to the bounding box
cropped_img = masked_img[y:y+h, x:x+w]

# Resize the cropped image to the target size
resized_img = cv2.resize(cropped_img, target_size)

# Display the original and cropped images
cv2.imshow('Original Image', img)
cv2.imshow('Cropped Image', cropped_img)
cv2.imshow('Resized Image', resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

2023-03-15 21:06:24.998 Python[98552:4036197] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/fj/r2kb_f4d3k1gxmcwsdc81_1r0000gn/T/org.python.python.savedState


In [2]:
img = cv2.imread("/Users/rishabhsharma/Documents/GitHub/rfmd/data/Training_set_processed/1.png")

# Convert the image to grayscale
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Resize the image to a standard size (e.g., 224x224 for VGG16)
resized_img = cv2.resize(gray_img, (224, 224))

# Convert the image to a NumPy array
img_array = np.array(resized_img)

# Add an extra dimension to the array to represent the "batch" size
img_array = np.expand_dims(img_array, axis=0)

# Normalize the pixel values to be between 0 and 1
img_array = img_array / 255.0

In [4]:
img_array.shape

(1, 224, 224)

In [5]:
import cv2
import numpy as np
import os
import pandas as pd

# Set the folder path containing images
folder_path = "/Users/rishabhsharma/Documents/GitHub/rfmd/data/Training_set_processed"

# Initialize an empty list to store image arrays and filenames
img_arrays = []
filenames = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (optional)
    if not filename.endswith(".jpg"):
        continue
    img = cv2.imread(os.path.join(folder_path, filename))
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (224, 224))
    img_array = np.array(resized_img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    img_arrays.append(img_array)
    filenames.append(filename)

# Concatenate all image arrays into a single NumPy array
img_arrays = np.concatenate(img_arrays, axis=0)

# Create a pandas DataFrame from the image arrays and filenames
df = pd.DataFrame({
    "Filename": filenames,
    "Image Array": img_arrays.tolist()
})


ValueError: need at least one array to concatenate